In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from src.models.train_model import train_with_loss
from src.models.train_model import random_splits

from src.models.gcn import GCN_fix_2layer

from src.models.reg import make_preg_ce_ce
from src.models.reg import make_preg_ce_ce_alt
from src.models.reg import make_abduls_preg_loss
from src.models.reg import make_lap_loss_ce
from src.models.reg import compute_a_hat

from src.models.evaluate_model import evaluate1

import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)

A_hat = compute_a_hat(data)

# data = random_splits(data, 50, 20)
data.reg_mask = torch.ones_like(data.train_mask, dtype=torch.bool)


In [6]:
print('-------------------------------------------------------------')
print(f'train size: {data.train_mask.sum()}')
print(f'val size: {data.val_mask.sum()}')
print(f'test size: {data.test_mask.sum()}')
print('-------------------------------------------------------------')

metrics = []

for mu in [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.]:
    torch.manual_seed(1)
    random.seed(1)

    loss_fn = make_preg_ce_ce(mu, A_hat)
    
    model = GCN_fix_2layer()

    model = train_with_loss(model, data, loss_fn, num_epochs=200)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'loss_fn_factory': 'cece', 'mu': mu, 'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
    print(metrics[-1])

for mu in [0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.]:
    # torch.manual_seed(1)
    # random.seed(1)

    loss_fn = make_preg_ce_ce_alt(mu, A_hat)
    
    model = GCN_fix_2layer()

    model = train_with_loss(model, data, loss_fn, num_epochs=200)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'loss_fn_factory': 'cecealt', 'mu': mu, 'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
    print(metrics[-1])

for mu in [0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.]:
    # torch.manual_seed(1)
    # random.seed(1)

    loss_fn = make_abduls_preg_loss(mu, A_hat)
    
    model = GCN_fix_2layer()

    model = train_with_loss(model, data, loss_fn, num_epochs=200)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'loss_fn_factory': 'abduls', 'mu': mu, 'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
    print(metrics[-1])


-------------------------------------------------------------
train size: 140
val size: 500
test size: 1000
-------------------------------------------------------------
{'loss_fn_factory': 'cece', 'mu': 0, 'train_acc': 1.0, 'val_acc': 0.784, 'test_acc': 0.8}
{'loss_fn_factory': 'cece', 'mu': 0.1, 'train_acc': 1.0, 'val_acc': 0.794, 'test_acc': 0.817}
{'loss_fn_factory': 'cece', 'mu': 0.2, 'train_acc': 0.9929, 'val_acc': 0.802, 'test_acc': 0.822}
{'loss_fn_factory': 'cece', 'mu': 0.3, 'train_acc': 0.9929, 'val_acc': 0.802, 'test_acc': 0.821}
{'loss_fn_factory': 'cece', 'mu': 0.4, 'train_acc': 0.9929, 'val_acc': 0.806, 'test_acc': 0.819}
{'loss_fn_factory': 'cece', 'mu': 0.5, 'train_acc': 0.9929, 'val_acc': 0.806, 'test_acc': 0.821}
{'loss_fn_factory': 'cece', 'mu': 0.6, 'train_acc': 0.9929, 'val_acc': 0.802, 'test_acc': 0.821}
{'loss_fn_factory': 'cece', 'mu': 0.7, 'train_acc': 0.9929, 'val_acc': 0.722, 'test_acc': 0.768}
{'loss_fn_factory': 'cece', 'mu': 0.8, 'train_acc': 0.9571, 'val

In [5]:
for mu in [0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.]:
    torch.manual_seed(1)
    random.seed(1)

    loss_fn = make_lap_loss_ce(mu)
    
    model = GCN_fix_2layer()

    model = train_with_loss(model, data, loss_fn, num_epochs=200)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'loss_fn_factory': 'lap', 'mu': mu, 'train_acc': np.round(train_acc,4), 'val_acc': np.round(val_acc,4), 'test_acc': np.round(test_acc,4)})
    print(metrics[-1])


{'loss_fn_factory': 'lap', 'mu': 0, 'train_acc': 1.0, 'val_acc': 0.784, 'test_acc': 0.8}
{'loss_fn_factory': 'lap', 'mu': 0.1, 'train_acc': 0.9857, 'val_acc': 0.628, 'test_acc': 0.649}
{'loss_fn_factory': 'lap', 'mu': 0.2, 'train_acc': 0.9857, 'val_acc': 0.586, 'test_acc': 0.585}
{'loss_fn_factory': 'lap', 'mu': 0.3, 'train_acc': 0.9857, 'val_acc': 0.586, 'test_acc': 0.58}
{'loss_fn_factory': 'lap', 'mu': 0.4, 'train_acc': 0.9786, 'val_acc': 0.586, 'test_acc': 0.573}
{'loss_fn_factory': 'lap', 'mu': 0.5, 'train_acc': 0.9786, 'val_acc': 0.586, 'test_acc': 0.572}
{'loss_fn_factory': 'lap', 'mu': 0.6, 'train_acc': 0.9786, 'val_acc': 0.578, 'test_acc': 0.577}
{'loss_fn_factory': 'lap', 'mu': 0.7, 'train_acc': 0.9786, 'val_acc': 0.572, 'test_acc': 0.577}
{'loss_fn_factory': 'lap', 'mu': 0.8, 'train_acc': 0.9786, 'val_acc': 0.578, 'test_acc': 0.586}
{'loss_fn_factory': 'lap', 'mu': 0.9, 'train_acc': 0.9786, 'val_acc': 0.576, 'test_acc': 0.585}
{'loss_fn_factory': 'lap', 'mu': 1.0, 'train_acc

In [6]:
df = pd.DataFrame(metrics)